# Задание 1
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание 2
* Выберите регион
* Сравните среднюю цену квадратного метра в городах этого региона
* Определите количество строящихся объектов и квартир в регионах
* Определите объем строящегося жилья в квадратных метрах

# Задание 3 *
* Напишете скрипт, который сможет ежедневно собирать данные по цене, количестве и объему строящихся объектов самостоятельно и ежедневно
* Попробуйте ускорить процесс
* Добавьте возможность добавления данных в БД с указанием текущей даты.

In [119]:
import requests
import pandas as pd
import numpy as np
import sqlite3
from tqdm.auto import tqdm
from geopy.geocoders import Nominatim
import time

## Список объектов

In [82]:
offset_ = 0
limit_ = 1000

result_df = pd.DataFrame()

url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
number_of_objects = pd.json_normalize(res.json()) ['data.total'][0]
number_of_loops = int(np.ceil(number_of_objects / 1000))

for i in range(number_of_loops):
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)

    objects_data = pd.json_normalize(pd.json_normalize(res.json())['data.list'][0])
    
    result_df = pd.concat([result_df,objects_data])
    
    offset_+=1000
    i+=1


10049
   objId  rpdRegionCd                          objAddr  objFloorMin  \
0  13391           72         г Тюмень, ул Профсоюзная            3   
1  13397           72  г Тюмень, ул Профсоюзная, д. 56            6   
2  13383           72                         г Тюмень            4   
3  13398           72  г Тюмень, ул Профсоюзная, д. 56            7   
4  13399           72  г Тюмень, ул Профсоюзная, д. 56            1   

   objFloorMax  objElemLivingCnt objReady100PercDt  objSquareLiving  \
0           12                44        2022-06-30          4293.67   
1           12                65        2024-12-31          6248.51   
2           20                68        2022-06-30          6289.98   
3           10                43        2024-12-31          4812.98   
4            1                 0        2024-12-31             0.00   

   objCompensFundFlg  objProblemFlg  ...  developer.orgForm.fullForm  \
0                  1              0  ...        Акционерное общество

In [83]:
#проверяем число строк
len(result_df)

10049

In [84]:
#проверяем собранные поля
result_df.keys()

Index(['objId', 'rpdRegionCd', 'objAddr', 'objFloorMin', 'objFloorMax',
       'objElemLivingCnt', 'objReady100PercDt', 'objSquareLiving',
       'objCompensFundFlg', 'objProblemFlg', 'latitude', 'longitude',
       'problemFlag', 'siteStatus', 'objGuarantyEscrowFlg',
       'residentialBuildings', 'rpdNum', 'pdId', 'buildType', 'objStatus',
       'developer.devId', 'developer.shortName', 'developer.fullName',
       'developer.groupName', 'developer.orgForm.id',
       'developer.orgForm.fullForm', 'developer.orgForm.shortForm',
       'developer.companyGroup', 'developer.devInn', 'objCommercNm',
       'complexShortName', 'developer.bankruptStage.bankruptStageCd',
       'developer.bankruptStage.bankruptStageDesc',
       'developer.bankruptStage.bankruptLawUrl',
       'developer.orgBankruptMsgDttm'],
      dtype='object')

In [85]:
#для дальнейшего сбора нужны будут только идентификаторы
ids = result_df.objId

# Данные по определенному объекту

In [93]:
#собираем подробную информацию в цикле
details_df = pd.DataFrame()

for object_id in tqdm(ids):
    url = url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{object_id}'

    res = requests.get(url)

    objects_data = pd.json_normalize(res.json())

    details_df = pd.concat([details_df,objects_data])
    

100%|███████████████████████████████████| 10049/10049 [1:07:19<00:00,  2.49it/s]

10049
<class 'pandas.core.frame.DataFrame'>


In [121]:
#проверяем собранные поля
print(details_df.columns.tolist())

['errcode', 'data.id', 'data.pdId', 'data.developer.devId', 'data.developer.devShortCleanNm', 'data.developer.devShortNm', 'data.developer.devFullCleanNm', 'data.developer.problObjCnt', 'data.developer.buildObjCnt', 'data.developer.comissObjCnt', 'data.developer.regRegionDesc', 'data.developer.devPhoneNum', 'data.developer.devSite', 'data.developer.devEmail', 'data.developer.devInn', 'data.developer.devOgrn', 'data.developer.devKpp', 'data.developer.devLegalAddr', 'data.developer.devFactAddr', 'data.developer.lastRpdId', 'data.developer.fundGuarantyFlg', 'data.developer.devOrgRegRegionCd', 'data.developer.devEmplMainFullNm', 'data.developer.developerGroupName', 'data.developer.orgForm.id', 'data.developer.orgForm.fullForm', 'data.developer.orgForm.shortForm', 'data.developer.companyGroupId', 'data.developer.objGuarantyEscrowFlg', 'data.developer.govFundFlg', 'data.region', 'data.address', 'data.floorMin', 'data.floorMax', 'data.objElemLivingCnt', 'data.objReady100PercDt', 'data.wallMat

## Запись

In [94]:
#сохраняем в excel
details_df.to_excel("dmrf.xlsx",sheet_name='Лист 1')

In [95]:
#сохраняем в pickle
details_df.to_pickle("dmrf.pkl")

In [111]:
#сохраняем в sqlite3
conn = sqlite3.connect(':memory:')

#при записи были ошибки с типами с тремя полями, поэтому переводим их в строковый тип
details_df['data.photoRenderDTO'] = details_df['data.objGuarantyEscrowFlg'].apply(str)
details_df['data.objectTransportInfo'] = details_df['data.objectTransportInfo'].apply(str)
details_df['data.metro.colors'] = details_df['data.metro.colors'].apply(str)

details_df.to_sql(name='dmrf', con=conn)

#проверяем
pd.read_sql('select * from dmrf', conn)

10049

,index,errcode,data.id,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,...,data.metro.colors,data.transportDistIndex,data.transportDistIndexValue,data.greenAreaIndexValue,data.airQualityIndexValue,data.complexShortNm,data.developer.bankruptStage.bankruptStageCd,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm
0,0,0,13391,28101,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,nan,None,NaN,NaN,NaN,None,NaN,None,None,None
1,0,0,13397,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,nan,None,NaN,NaN,NaN,None,NaN,None,None,None
2,0,0,13383,28105,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,nan,None,NaN,NaN,NaN,None,NaN,None,None,None
3,0,0,13398,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,nan,None,NaN,NaN,NaN,None,NaN,None,None,None
4,0,0,13399,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,...,nan,None,NaN,NaN,NaN,None,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10044,0,0,28071,99284,7344,ЯРКИЙ МИР,ЯРКИЙ МИР,ЯРКИЙ МИР,0,5,...,['#943E90'],None,NaN,NaN,8.0,None,NaN,None,None,None
10045,0,0,28070,99487,7344,ЯРКИЙ МИР,ЯРКИЙ МИР,ЯРКИЙ МИР,0,5,...,['#943E90'],None,NaN,NaN,7.0,None,NaN,None,None,None
10046,0,0,44447,475093,2538,ЯРОСЛАВЛЬЗАКАЗЧИК,ЯРОСЛАВЛЬЗАКАЗЧИК,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК ЯРОСЛАВЛЬЗАКАЗЧИК,0,1,...,nan,None,NaN,NaN,NaN,None,NaN,None,None,None
10047,0,0,28000,91796,7339,ЯРУС,ЯРУС,ЯРУС,0,1,...,nan,None,NaN,NaN,NaN,None,NaN,None,None,None


## Задание 2

In [124]:
#можно использовать код от Никиты для определения региона, но можно и использовать поле data.region
locations = []

for la, lo in tqdm(zip(list(details_df['data.objLkLatitude']), list(details_df['data.objLkLongitude']))):
    geolocator = Nominatim(user_agent="http")
    try:
        location = geolocator.reverse(f"{la}, {lo}")
        locations.append(location.raw['address']['state'])
    except:
        locations.append('Not Detected')
    time.sleep(0.25)

details_df['location'] = locations

#spb_df = details_df[details_df['data.developer.regRegionDesc'] == 'Санкт-Петербург город']


10049it [5:17:42,  1.90s/it]


In [149]:
details_df[details_df['location'] == 'Ленинградская область']['data.address'].head(10)

0                                          д Агалатово
0          гп Янино-1, улица 7-ая линия, вл. Участок 1
0    г Сертолово, Микрорайон Сертолово-1, улица Ларина
0                                       г Сосновый Бор
0                                   п Бугры, корпус 22
0                                   п Бугры, корпус 24
0                                   п Бугры, корпус 25
0                                   п Бугры, корпус 23
0                                   п Бугры, корпус 26
0    с Русско-Высоцкое, район АОЗТ «Птицефабрика Ру...
Name: data.address, dtype: object

In [151]:
# сформируем отдельные датасеты
yanino = details_df[details_df['data.address'].str.contains("Янино", na=False)]
bugry = details_df[details_df['data.address'].str.contains("Бугры", na=False)]

In [154]:
#число объектов
print('yanino:', len(yanino))
print('bugry:', len(bugry))

yanino: 25
bugry: 36


In [156]:
#средняя стоимость
print('yanino:', yanino['data.objPriceAvg'].mean())
print('bugry:', bugry['data.objPriceAvg'].mean())

yanino: 111206.4
bugry: 122223.48148148147


'data.objPriceAvg'

In [157]:
#объем жилья
print('yanino:', yanino['data.objSquareLiving'].sum())
print('bugry:', bugry['data.objSquareLiving'].sum())

yanino: 210379.57
bugry: 448378.27999999997
